In [1]:
#pip install scikit-learn

In [2]:
# pip install pydotplus (uncomment to run install)


In [3]:
# pip install numpy scipy matplotlib

In [4]:
# pip install graphviz

In [5]:

# Import libraries for data analysis and visualization
import pandas as pd 
import numpy as np
from  sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 

# Needed for decision tree visualization 
# Install Pydotplus 
import pydotplus
from IPython.display import Image
import graphviz



In [6]:
# Import the data
sp500_adj_close = "sp500_adj_close_raw_vick.csv"
sp500_df = pd.read_csv(sp500_adj_close)
sp500_df.head()


,Date,Ticker,Adjusted Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2023-10-24 00:00:00+00:00,A,104.928497,-0.021204,0.409553,56.101762,121.677983,128.300623,133.159750,115.267512,98.748279,100.488609,138.878296,buy
1,2023-10-24 00:00:00+00:00,AAPL,172.557007,-0.013492,0.188448,72.685668,186.475519,184.511592,189.937574,194.716172,162.902436,166.040359,197.361084,short
2,2023-10-24 00:00:00+00:00,ABBV,141.150238,-0.007177,0.208797,31.460189,140.884359,154.118351,160.623434,141.142258,130.635863,132.747406,155.765976,short
3,2023-10-24 00:00:00+00:00,ABNB,122.010002,-0.028768,0.456553,45.172589,130.258200,140.167450,145.392425,130.042030,111.038970,114.089996,147.500000,short
4,2023-10-24 00:00:00+00:00,ABT,92.954971,-0.013079,0.170162,45.183485,100.178232,106.883077,106.050604,98.488926,89.118164,91.033325,108.808609,buy


In [7]:
sp500_df.set_index(['Date','Ticker'], inplace=True)
sp500_df.head()

Adjusted Close    Return  Volatility  \
Date                      Ticker                                         
2023-10-24 00:00:00+00:00 A           104.928497 -0.021204    0.409553   
                          AAPL        172.557007 -0.013492    0.188448   
                          ABBV        141.150238 -0.007177    0.208797   
                          ABNB        122.010002 -0.028768    0.456553   
                          ABT          92.954971 -0.013079    0.170162   

                                        RSI      SMA_50     SMA_100  \
Date                      Ticker                                      
2023-10-24 00:00:00+00:00 A       56.101762  121.677983  128.300623   
                          AAPL    72.685668  186.475519  184.511592   
                          ABBV    31.460189  140.884359  154.118351   
                          ABNB    45.172589  130.258200  140.167450   
                          ABT     45.183485  100.178232  106.883077   

                                     SMA_200  Upper Band  Lower Band  \
Date                      Ticker                                       
2023-10-24 00:00:00+00:00 A       133.159750  115.267512   98.748279   
                          AAPL    189.937574  194.716172  162.902436   
                          ABBV    160.623434  141.142258  130.635863   
                          ABNB    145.392425  130.042030  111.038970   
                          ABT     106.050604   98.488926   89.118164   

                                     Support  Resistance Action  
Date                      Ticker                                 
2023-10-24 00:00:00+00:00 A       100.488609  138.878296    buy  
                          AAPL    166.040359  197.361084  short  
                          ABBV    132.747406  155.765976  short  
                          ABNB    114.089996  147.500000  short  
                          ABT      91.033325  108.808609    buy

In [8]:
# Define features (everything except the "Ticker" column)
X = sp500_df.copy()
X = X.drop(["Action","Return"], axis=1)
X.head()

Adjusted Close  Volatility        RSI  \
Date                      Ticker                                          
2023-10-24 00:00:00+00:00 A           104.928497    0.409553  56.101762   
                          AAPL        172.557007    0.188448  72.685668   
                          ABBV        141.150238    0.208797  31.460189   
                          ABNB        122.010002    0.456553  45.172589   
                          ABT          92.954971    0.170162  45.183485   

                                      SMA_50     SMA_100     SMA_200  \
Date                      Ticker                                       
2023-10-24 00:00:00+00:00 A       121.677983  128.300623  133.159750   
                          AAPL    186.475519  184.511592  189.937574   
                          ABBV    140.884359  154.118351  160.623434   
                          ABNB    130.258200  140.167450  145.392425   
                          ABT     100.178232  106.883077  106.050604   

                                  Upper Band  Lower Band     Support  \
Date                      Ticker                                       
2023-10-24 00:00:00+00:00 A       115.267512   98.748279  100.488609   
                          AAPL    194.716172  162.902436  166.040359   
                          ABBV    141.142258  130.635863  132.747406   
                          ABNB    130.042030  111.038970  114.089996   
                          ABT      98.488926   89.118164   91.033325   

                                  Resistance  
Date                      Ticker              
2023-10-24 00:00:00+00:00 A       138.878296  
                          AAPL    197.361084  
                          ABBV    155.765976  
                          ABNB    147.500000  
                          ABT     108.808609

In [9]:
# Checkingn the unique value of the Return
sp500_df['Return'].unique()

array([-0.02120403, -0.01349165, -0.00717654, ..., -0.01034708,
       -0.00815021, -0.0195054 ])

In [10]:
# Defining y
y = sp500_df['Return'].values.reshape(-1,1)
y[:3]

array([[-0.02120403],
       [-0.01349165],
       [-0.00717654]])

In [11]:
# Use train_test_split to separate the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [13]:
# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Once data is scaled, created a decision tree instances and trained it with the (X_trained_scaled and Y_train)

In [15]:
# Cretad the decistion tree classifier instance 
model = tree.DecisionTreeRegressor()

In [16]:
model = model.fit(X_train_scaled, y_train)

Validate the trained model, by predicting sp500 action predictions using the (X_test_scaled). 

In [17]:
# Making prediction using the testing data 
predictions = model.predict(X_test_scaled)

## Model Evaluation
Evaluate model's results, by using 'sklearn' to calculate the necessary score.

In [18]:
# Checking the Mean Squared Error using mse 
mse = mean_squared_error(y_test, predictions)

print(f"Mean Squared Error : {mse}")

Mean Squared Error : 0.0006647584109640664


Visualization of the Data 

In [20]:
# How to creata a visualization of the decision tree regression model 
# Fix bug in this code

# Create DOT data
dot_data = tree.export_graphviz(
    model, out_file=None, feature_names=X.columns, class_names=["0", "1"], filled=True
)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png)

FileNotFoundError: No such file or directory: '<function Dot.__init__.<locals>.<lambda> at 0x49b1f3370>'

FileNotFoundError: No such file or directory: '<function Dot.__init__.<locals>.<lambda> at 0x49b1f3370>'

<IPython.core.display.Image object>